<a href="https://colab.research.google.com/github/Tania526-sudo/Image-Deblurring/blob/main/notebooks/Deblur_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from MPRNet import MPRNet

In [4]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 43.8 MB/s eta 0:00:00


In [3]:
import torch
def export_to_onnx(weights_path: str, onnx_path: str, input_size=(3, 256, 256)):
    device = torch.device('cpu')
    model = MPRNet()
    checkpoint = torch.load(weights_path, map_location=device)
    state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint

    #  state_dict (DataParallel)
    new_state_dict = {}
    for k, v in state_dict.items():
        name = k[7:] if k.startswith('module.') else k
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    model.to(device)
    model.eval()

    dummy_input = torch.randn(1, *input_size, device=device)
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        opset_version=11,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size', 2: 'height', 3: 'width'},
                      'output': {0: 'batch_size', 2: 'height', 3: 'width'}},
        verbose=True
    )
    print(f"Model exported to ONNX at {onnx_path}")

if __name__ == "__main__":
    export_to_onnx(
        weights_path="pretrained_models/model_deblurring.pth",
        onnx_path="model_deblurring.onnx",
        input_size=(3, 256, 256)
    )

/content/MPRNet.py:271: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x2top_img  = x3_img[:,:,0:int(H/2),:]
/content/MPRNet.py:272: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x2bot_img  = x3_img[:,:,int(H/2):H,:]
/content/MPRNet.py:275: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x1ltop_img = x2top_img[:,:,:,0:int(W/2)]
/content/M

Model exported to ONNX at model_deblurring.onnx


In [5]:
!pip install openvino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 20.7 MB/s eta 0:00:00


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install torch torchvision onnx onnxruntime openvino openvino-dev opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import torch
from MPRNet import MPRNet

def export_to_onnx(weights_path, onnx_path, input_size=(1,3,256,256)):
    model = MPRNet()
    checkpoint = torch.load(weights_path, map_location='cpu')
    state_dict = checkpoint.get('state_dict', checkpoint)
    new_state_dict = {k[7:] if k.startswith('module.') else k:v for k,v in state_dict.items()}
    model.load_state_dict(new_state_dict)
    model.eval()
    dummy_input = torch.randn(input_size)
    torch.onnx.export(model, dummy_input, onnx_path, opset_version=11,
                      input_names=['input'], output_names=['output'],
                      dynamic_axes={'input':{0:'batch_size'}, 'output':{0:'batch_size'}})
    print(f"Model exported to ONNX at {onnx_path}")

export_to_onnx("model_deblurring.pth", "model_deblurring.onnx")

/content/MPRNet.py:271: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x2top_img  = x3_img[:,:,0:int(H/2),:]
/content/MPRNet.py:272: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x2bot_img  = x3_img[:,:,int(H/2):H,:]
/content/MPRNet.py:275: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x1ltop_img = x2top_img[:,:,:,0:int(W/2)]
/content/M

Model exported to ONNX at model_deblurring.onnx


In [21]:
import onnxruntime as ort
import numpy as np
import cv2

def preprocess(image_path, input_size=(256, 256)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, input_size)
    img = img.astype('float32') / 255.0
    img = np.transpose(img, (2, 0, 1))  # HWC to CHW
    img = np.expand_dims(img, axis=0)  # add batch dimension
    return img

def postprocess(output_tensor):
    output_tensor = np.squeeze(output_tensor)
    output_tensor = np.clip(output_tensor, 0, 1)
    output_tensor = (output_tensor * 255).astype(np.uint8)
    output_tensor = np.transpose(output_tensor, (1, 2, 0))  # CHW to HWC
    output_tensor = cv2.cvtColor(output_tensor, cv2.COLOR_RGB2BGR)
    return output_tensor

# Запуск інференсу
session = ort.InferenceSession("model_deblurring.onnx")
input_name = session.get_inputs()[0].name

input_image = preprocess("blurred_images/GOPR0372_07_00-000049.png")
outputs = session.run(None, {input_name: input_image})

output_image = postprocess(outputs[0])
cv2.imwrite("results/deblurred_output.png", output_image)
print("Saved deblurred output image.")

Saved deblurred output image.


In [23]:
!pip install --upgrade openvino-dev


In [24]:
from openvino.tools.mo import convert_model

convert_model(
    "model_deblurring.onnx",
    output_dir="openvino_model",
    compress_to_fp16=True
)

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html


<Model: 'main_graph'
inputs[
<ConstOutput: names[input] shape[?,3,?,?] type: f32>
]
outputs[
<ConstOutput: names[output] shape[?,3,8..,8..] type: f32>,
<ConstOutput: names[input.1316] shape[?,3,8..,8..] type: f32>,
<ConstOutput: names[888] shape[?,3,8..,8..] type: f32>
]>

In [25]:
import os
print(os.listdir("openvino_model"))


['model_deblurring.onnx']
